[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_whisper_ja_video.ipynb)

In [ ]:
!pip install transformers gradio yt_dlp pytube

In [ ]:
# https://huggingface.co/spaces/vumichien/whisper-speaker-diarization/blob/main/app.py modifiyed

from transformers import pipeline
from pytube import YouTube

pipe = pipeline(
    task="automatic-speech-recognition",
    model="vumichien/whisper-medium-jp",
    chunk_length_s=30,
    device=0,
)

pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="ja", task="transcribe")

def _return_yt_html_embed(yt_url):
    video_id = yt_url.split("?v=")[-1]
    HTML_str = (
        f'<center> <iframe width="500" height="320" src="https://www.youtube.com/embed/{video_id}"> </iframe>'
        " </center>"
    )
    return HTML_str

def yt_transcribe(yt_url):
    yt = YouTube(yt_url)
    html_embed_str = _return_yt_html_embed(yt_url)
    stream = yt.streams.filter(only_audio=True)[0]
    stream.download(filename="audio.mp3")
    text = pipe("audio.mp3")["text"]
    return html_embed_str, text

In [ ]:
from yt_dlp import YoutubeDL
#with YoutubeDL({'overwrites':True, 'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4', 'outtmpl':'/content/video'}) as ydl:
with YoutubeDL({'overwrites':True, 'format':'bestaudio[ext=m4a]', 'outtmpl':'/content/audio.mp4'}) as ydl:
    ydl.download("https://www.youtube.com/watch?v=ulziqjIMDKQ")

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="vumichien/whisper-medium-jp", device=0,)
text = pipe("/content/audio.mp4")["text"]
print(text)